In [1]:
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


import time

In [2]:
productDF = pd.read_csv('./product.csv', index_col=0)

In [3]:
productDF.sample(5)
len(productDF)

122

## 🔍 어떤 쇼핑몰 URL 부터 Crawling 할 지?
####  후보 쇼핑몰(개인적인 우선순위)
1. 29cm
2. 카카오 선물하기
3. 쿠팡
4. Wconcept
5. 네이버 쇼핑, 스마트스토어
6. sivillage(신세계몰), lotte

In [4]:
shop_lst = ['29cm', 'gift.kakao', 'kko.to', 'coupang', 'wconcept','smartstore']

for url in productDF['productURL']:
    if not any(shop in url for shop in shop_lst): print(url)

https://www.kurly.com/goods/5094018
https://www.eqlstore.com/product/GPHA22012014994/detail
https://www.eqlstore.com/product/GPIT23021641279/detail
https://www.collectionb.cc/goods/goods_view.php?goodsNo=1000025641
https://www.le-jour.com/goods/goods_view.php?goodsNo=1000000046
https://www.le-jour.com/goods/goods_view.php?goodsNo=1000000046
https://makers.kakao.com/promotion/392
https://www.tamburins.com/kr/item/1666769254/
https://www.sivillage.com/goods/initDetailGoods.siv?partnerNm=navershopping_pc&goods_no=2204491100&utm_source=navershopping&utm_medium=cps&utm_term=%7Bquery%7D&NaPm=ct%3Dle3qyxig%7Cci%3D4e3c99125a5325130bfbe68c6d1e029a19f4ca02%7Ctr%3Dslsbrc%7Csn%3D490979%7Chk%3Dee07948f0b2311500b37b1c1836f3be0dbbbd7de
https://www.lotteon.com/p/product/LE1204196371?sitmNo=LE1204196371_1222410981&dp_infw_cd=CAT43401&areaCode=CAT5004
https://www.hmall.com/pd/pda/itemPtc?ReferCode=429&slitmCd=2061290505&utm_source=naver&utm_medium=cps_pcs&utm_campaign=sale&NaPm=ct%3Dle3sxsfs%7Cci%3D816a

**쇼핑몰 리스트**
1. 29cm
2. 카카오 선물하기(gift.kakao, kko.to), 카카오 메이커스(makers.kakao)->버려
3. 쿠팡
4. Wconcept
5. 네이버 쇼핑, 스마트스토어
6. sivillage(신세계몰), lotteon, hmall(현대)
7. oliveyoung
8. eqlstore(한섬), collectionb(오브제 컬렉션)
9. kurly
10. 브랜드 공홈 : le-jour, tamburins, jomalone, dior, nike, channel


In [5]:
product_shoppingmall = []
for url in productDF['productURL']:
    if '29cm' in url : product_shoppingmall.append('29cm')
    elif any(shop in url for shop in ['gift.kakao', 'kko.to']): product_shoppingmall.append('카카오 선물하기')
    elif 'coupang' in url: product_shoppingmall.append('쿠팡')
    elif 'wconcept' in url: product_shoppingmall.append('wconcept')
    elif 'smartstore' in url: product_shoppingmall.append('네이버')
    elif any(shop in url for shop in ['sivillage', 'lotteon', 'hmall']): product_shoppingmall.append('백화점/홈쇼핑 몰')
    elif 'oliveyoung' in url: product_shoppingmall.append('올리브영')
    elif any(shop in url for shop in ['eqlstore', 'collecionb', 'ohou.se']): product_shoppingmall.append('인테리어/오브제 몰')
    elif 'kurly' in url: product_shoppingmall.append('마켓컬리')
    elif 'wconcept' in url: product_shoppingmall.append('29cm')
    elif 'makers.kakao' in url: product_shoppingmall.append('')
    else: product_shoppingmall.append('브랜드 공식 홈페이지')

productDF['shoppingmall'] = product_shoppingmall
productDF

,videoID,productURL,shoppingmall
0,mLDpy3Pv_XU,https://content.29cm.co.kr/post/41215,29cm
1,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/1814100,29cm
2,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/2209304,29cm
3,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/1970379,29cm
4,mLDpy3Pv_XU,https://product.29cm.co.kr/catalog/1671349,29cm
...,...,...,...
117,JWRlTxyS0c8,https://gift.kakao.com/product/6983306,카카오 선물하기
118,JWRlTxyS0c8,https://gift.kakao.com/product/4938088,카카오 선물하기
119,SDOVtmEqVGE,https://gift.kakao.com/product/6780518?input_c...,카카오 선물하기
120,SDOVtmEqVGE,https://makers.kakao.com/promotion/392,


In [6]:
# 카카오 메이커스는 컬렉션 링크 -> not 제품링크
productDF = productDF[productDF['shoppingmall'] != '']
productDF
productDF.reset_index(drop=True, inplace=True)


In [7]:
productDF['shoppingmall'].value_counts()

shoppingmall
카카오 선물하기       59
29cm           34
브랜드 공식 홈페이지    11
백화점/홈쇼핑 몰       7
wconcept        4
인테리어/오브제 몰      2
마켓컬리            1
올리브영            1
Name: count, dtype: int64

### 1️⃣ 카카오 선물하기 제품 정보 crawling

[카카오 api 활용](https://shopping-developers.kakao.com/hc/ko/articles/4622557003151-2-%EC%B9%B4%EC%B9%B4%EC%98%A4%EA%B3%84%EC%A0%95-2%EA%B0%9C-%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0)
-> 판매자 등록하는 방식 (사업자등록증 필요)

In [ ]:
kkoDF = productDF[productDF['shoppingmall']=='카카오 선물하기']
kkoDF.reset_index(inplace=True)
kkoDF.columns = ['productID', 'videoID', 'productURL', 'shoppingmall']
len(kkoDF)

59

In [ ]:
# URL 에 brand 가 들어가면 카탈로그가 나옴
# https://gift.kakao.com/product/id의 형태가 아니면 제품 정보가 아님
drop_idx = []
for idx in range(len(kkoDF['productURL'])):
    url = kkoDF['productURL'][idx]
    if not re.match('https://gift.kakao.com/product', url):
        print(kkoDF['productURL'][idx])
        drop_idx.append(idx)

kkoDF = kkoDF.drop(drop_idx)
kkoDF.reset_index(inplace=True, drop=True)
len(kkoDF)

https://gift.kakao.com/brand/11232
https://gift.kakao.com/page/15951
https://gift.kakao.com/brand/14586?display=basic
https://gift.kakao.com/page/15951
https://gift.kakao.com/page/15951


54

In [ ]:
# print(kkoDF['productURL'][38])
# kkoDF.loc[38]
kkoDF['productURL'].loc[38] = kkoDF['productURL'][38][:len(kkoDF['productURL'][38])//2]
print(kkoDF['productURL'][38])

https://gift.kakao.com/product/6836524


/var/folders/sp/rr963h912c1c9rvvd41sr4_h0000gn/T/ipykernel_1032/1047807000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kkoDF['productURL'].loc[38] = kkoDF['productURL'][38][:len(kkoDF['productURL'][38])//2]


In [ ]:
# 상품 번호 뒤로는 url 자르기
short_url = []
i =0
for url in kkoDF['productURL']:
    cut_idx = url.find('?')
    if cut_idx == -1:
        short_url.append(url)
    else:
        short_url.append(url[:cut_idx])
    
kkoDF['shortURL'] = short_url
print(len(kkoDF))
kkoDF.drop_duplicates(subset='shortURL', keep='first')
print(len(kkoDF))

54
54


In [ ]:
def get_kko_product_info(url):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome() 
    driver.get(url + '?tab=detail') 

    # 필요한 값이 로드될 때 까지 기다림
    try : 
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.txt_price'))
        )
    except TimeoutException as e:
        driver.quit()
        return None


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 값 가져오기

    # 1. 상품명
    product_name = soup.select_one('h2.tit_subject').get_text()
    # 2. 가격
    if soup.select_one('span.txt_sale'):
        discount_rate = soup.select_one('span.txt_sale').get_text()
        price = soup.select_one('div.info_product.clear_g').select_one('span.txt_price').get_text()
        discount_price = soup.select_one('div.info_product.clear_g').select_one('span.txt_total').get_text()
    else:
        discount_price = soup.select_one('div.info_product.clear_g').select_one('span.txt_total').get_text()
        price = discount_price
        discount_rate = '0%'
        
    # 3. 브랜드 정보
    brand_name = soup.select_one('span.txt_shopname').get_text()
    brand_link_inshop = 'https://gift.kakao.com' + soup.select_one('a.link_shopname')['href']

    product_info_table = dict(product_name=product_name, 
                              price=price, discount_price=discount_price,# discount_rate=discount_rate,
                              brand_name=brand_name,brand_link_inshop=brand_link_inshop)
    # 4. 상품상세 정보 표
    product_info_lst= soup.select_one('table.tbl_detail').select('tr')
    for product_info in product_info_lst:
        category_name , category_val = product_info.select_one('th').get_text(), product_info.select_one('td').get_text()
        if category_val == '상품상세설명 참조': continue
        product_info_table[category_name] = category_val


    # 불러온 값이 None 일 때의 대비 아직 미완

    # 브라우저 닫기
    driver.quit()

    return product_info_table


In [ ]:
kkoDF['productInfo'] = kkoDF['shortURL'].apply(get_kko_product_info)

In [ ]:
kkoDF.to_csv('./productInfo/kko.csv')

### 2️⃣ 29cm 제품 정보 crawling


In [8]:
cmDF = productDF[productDF['shoppingmall']=="29cm"]
cmDF.reset_index(inplace=True)
cmDF.columns = ['productID', 'videoID', 'productURL', 'shoppingmall']
len(cmDF)

34

In [128]:
# 상품 번호 뒤로는 url 자르기
short_url = []
i =0
for url in cmDF['productURL']:
    cut_idx = url.find('?')
    if cut_idx == -1:
        short_url.append(url)
    else:
        print(url)
        short_url.append(url[:cut_idx])
        
cmDF['shortURL'] = short_url
print(len(cmDF))
cmDF.drop_duplicates(subset='shortURL', keep='first')
cmDF.reset_index(drop=True, inplace=True)
print(len(cmDF))

https://product.29cm.co.kr/catalog/2047058?source=brand
https://product.29cm.co.kr/catalog/1967440?source=brand
https://product.29cm.co.kr/catalog/1662994?source=like&source_tab=product
https://product.29cm.co.kr/catalog/1798319?source=like&source_tab=product
https://product.29cm.co.kr/catalog/1759079?source=like&source_tab=product
https://product.29cm.co.kr/catalog/1759102?source=like&source_tab=product
https://product.29cm.co.kr/catalog/1803839?source=like&source_tab=product
https://product.29cm.co.kr/catalog/791401?source=brand
34
34


/var/folders/sp/rr963h912c1c9rvvd41sr4_h0000gn/T/ipykernel_1032/4204268944.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmDF['shortURL'] = short_url


In [132]:
drop_idx = [] 
for i in range(len(cmDF)):
    if not any(item in cmDF['shortURL'][i] for item in ['product', 'catalog']): 
        drop_idx.append(i)
        print(cmDF['shortURL'][i])

print(len(cmDF))
cmDF.drop(labels=drop_idx, inplace=True)
print(len(cmDF))
cmDF.reset_index(inplace=True, drop=True)

https://content.29cm.co.kr/post/41215
34
33


/var/folders/sp/rr963h912c1c9rvvd41sr4_h0000gn/T/ipykernel_1032/999922547.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmDF.drop(labels=drop_idx, inplace=True)


In [184]:
def get_29cm_product_info(url):
    # Selenium WebDriver 설정
    driver = webdriver.Chrome() 
    driver.get(url) 

    # 필요한 값이 로드될 때 까지 기다림
    try : 
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.css-4bcxzt.ejuizc34'))
        )
    except TimeoutException as e:
        driver.quit()
        return None


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 값 가져오기

    # 1. 상품명
    product_name = soup.select_one('#pdp_product_name').get_text().strip()
    # 2. 가격
    if soup.select_one('p.css-1bci2fm.ejuizc31'): # 할인할 때
        price = soup.select_one('p.css-1bci2fm.ejuizc31').get_text().strip()
        discount_price = soup.select_one('span.css-4bcxzt.ejuizc34').get_text().strip()
        discount_rate = soup.select_one('span.css-1jsmahk.ejuizc32').get_text().strip()
    else: # 할인 안할 때
        price = soup.select_one('span.css-4bcxzt.ejuizc34').get_text().strip()
        discount_price = price
        discount_rate = '0%'
    # 3. 브랜드 정보
    brand_name = soup.select_one('h3.css-1dncbyk.e1kth5844').get_text().strip()
    brand_description = soup.select_one('p.css-8e7eit.e1kth5845').get_text().strip()
    brand_link_inshop = soup.select_one('a.css-k95f3n.e1kth5841')['href']

    product_info_table = dict(product_name=product_name, 
                              price=price, discount_price=discount_price, discount_rate=discount_rate,
                              brand_name=brand_name,brand_description=brand_description,brand_link_inshop=brand_link_inshop)
    # 4. 상품상세 정보 표
    product_info_lst= soup.select_one('table.e1hw6jas2.css-1x7jfi1.exbpx9h0').select('tr')
    for product_info in product_info_lst:
        category_name , category_val = product_info.select_one('th').get_text().strip(), product_info.select_one('td').get_text().strip()
        if category_val == '상세페이지 참조': continue
        product_info_table[category_name] = category_val


    # 불러온 값이 None 일 때의 대비 아직 미완

    # 브라우저 닫기
    driver.quit()

    return product_info_table


In [186]:
cmDF['productInfo'] = cmDF['shortURL'].apply(get_29cm_product_info)

/var/folders/sp/rr963h912c1c9rvvd41sr4_h0000gn/T/ipykernel_1032/3804479893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmDF['productInfo'] = cmDF['shortURL'].apply(get_29cm_product_info)


In [188]:
cmDF.to_csv('./productInfo/29cm.csv')

## 29cm + 카카오톡 선물하기 제품 정보 합치기


In [326]:
totProductInfo = pd.concat([kkoDF, cmDF])
totProductInfo.reset_index(drop=True, inplace=True)    
len(totProductInfo)

87

In [327]:
drop_idx = []
for i in range(len(totProductInfo)):
    if not totProductInfo['productInfo'][i]:
        drop_idx.append(i)

totProductInfo.drop(labels=drop_idx, inplace=True)
totProductInfo.reset_index(drop=True, inplace=True)
len(totProductInfo)

84

In [328]:
info_lst = []
info_dict = dict()
for info in totProductInfo['productInfo']:
    if not info: continue
    info_lst.extend(list(info.keys()))


for i, val in enumerate(info_lst):
    pattern = r'[^a-zA-Z0-9가-힣,_]'
    key = re.sub(pattern, '', val)
    if key in info_dict.keys():
        if val in info_dict[key]: continue
        else: info_dict[key].append(val)
    else: info_dict[key] = [val]

info_dict.keys()


dict_keys(['product_name', 'price', 'discount_price', 'brand_name', 'brand_link_inshop', '내용물의용량또는중량', '사용기한또는개봉후사용기간', '화장품제조업자,화장품책임판매업자및맞춤형화장품판매업자', '제조국', '화장품법에따라기재표시하여야하는모든성분', '화장품법에따른기능성화장품미백,주름개선,자외선차단제품등의경우화장품법에따른기능성화장품심사또는보고를필함의문구', '사용할때주의사항', '품질보증기준', '소비자상담관련전화번호', '제품주요사양', '사용방법', '수입품의경우생산자,수입자및제조국', '제조연월일,소비기한또는품질유지기한', '유전자변형식품에해당하는경우의표시', '수입식품의경우수입식품안전관리특별법에따른수입신고를필함의문구', '품명및모델명', '법에의한인증허가등을받았음을확인할수있는경우그에대한사항', '제조국또는원산지', '제조자', 'AS책임자와전화번호또는소비자상담관련전화번호', '제품소재', '색상', '치수', '제품구성', '세탁방법및취급시주의사항', 'AS책임자와전화번호', '재질', '구성품', '크기', '동일모델의출시년월', '수입식품안전관리특별법에따른수입기구또는용기포장의경우수입식품안전관리특별법에따른수입신고를필함의문구', 'KC인증정보', '정격전압,소비전력', '크기,무게', '주요사양', '품목및제품명', '용도및제형', '에너지소비효율등급', '크기,형태', '화면사양', '추가설치비용', '종류', '소재', '식품의유형', '생산자및소재지', '포장단위별내용물의용량중량,수량', '원재료명및함량', '영양성분', '소비자안전을위한주의사항', '제품명', 'discount_rate', 'brand_description', '수입식품', '제조업소의명칭과소재지', '소비기한및보관방법', '원료명및함량', '영양정보', '기능정보', '섭취량,섭취방법및섭취시주의사항및부작용가능성', '질병의예방및치료를위한의약품이아니라는내용의표현', '유전자변형건강기능식품표시', '수입건

In [329]:
# 제품 기본 정보 먼저 Table에 넣기
# : 제품명, 브랜드명, 브랜드 설명, 브랜드 링크, 가격(정가, 판매가, 할인율) 

product_name = []
original_price = []
current_price = []
discount_rate = []
brand_name = []
brand_description = []
brand_url_inshop = []

for info in totProductInfo['productInfo']:
     try:
          product_name.append(info['product_name'])
     except:
          product_name.append(None)
     try:
          current_price.append(info['discount_price'])
     except:
          current_price.append(None)
     try:
          original_price.append(info['price'])
     except:
          original_price.append(None)
     try:
          discount_rate.append(info['discount_rate'])
     except:
          discount_rate.append(None)
     try:
          brand_name.append(info['brand_name'])
     except:
          discount_rate.append(None)     
     try:
          brand_description.append(info['brand_description'])
     except:
          brand_description.append(None)
     try:
          brand_url_inshop.append(info['brand_link_inshop'])
     except:
          brand_url_inshop.append(None)

totProductInfo['product_name'] = product_name
totProductInfo['original_price'] = original_price
totProductInfo['current_price'] = current_price
totProductInfo['discount_rate'] = discount_rate
totProductInfo['brand_name'] = brand_name
totProductInfo['brand_description'] = brand_description
totProductInfo['brand_url_inshop'] = brand_url_inshop


In [ ]:
totProductInfo

In [ ]:
# 가격 정보 int로 변경 가능한 형태로 변경하기
def remove_non_numeric(input_string):
    # 숫자가 아닌 모든 문자를 찾는 정규 표현식 패턴
    pattern = r'[^0-9]'
    # 숫자가 아닌 모든 문자를 빈 문자열로 대체
    cleaned_string = re.sub(pattern, '', input_string)
    return int(cleaned_string)



totProductInfo['original_price'] = totProductInfo['original_price'].apply(remove_non_numeric)
totProductInfo['current_price'] = totProductInfo['current_price'].apply(remove_non_numeric)
totProductInfo

In [332]:
discount = []
for idx, rate in enumerate(totProductInfo['discount_rate']):
    if not rate:
        discount.append(int(100 - ((totProductInfo['current_price'][idx] * 100) / totProductInfo['original_price'][idx])))
    else:
        discount.append(remove_non_numeric(rate))
totProductInfo['discount_rate'] = discount
totProductInfo


,productID,videoID,productURL,shoppingmall,shortURL,productInfo,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_url_inshop
0,24,fflimV2pIL8,https://gift.kakao.com/product/6252042,카카오 선물하기,https://gift.kakao.com/product/6252042,{'product_name': ' [리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 ...,[리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 한 번에' 휩드 비건 팩클렌저 1...,22700,22700,0,휩드,None,https://gift.kakao.com/brand/14844
1,28,ONFJdkVFSzY,https://gift.kakao.com/product/6784177,카카오 선물하기,https://gift.kakao.com/product/6784177,{'product_name': ' 월레스와 그로밋 일회용 필름 카메라 - 화려한 외...,월레스와 그로밋 일회용 필름 카메라 - 화려한 외출,27900,27900,0,콜리,None,https://gift.kakao.com/brand/14472
2,29,ONFJdkVFSzY,https://gift.kakao.com/product/5627132,카카오 선물하기,https://gift.kakao.com/product/5627132,{'product_name': ' [단독+포장] 바디오일 & 샌달우드 괄사&에센셜오...,[단독+포장] 바디오일 & 샌달우드 괄사&에센셜오일 세트 / 3종 택1,29000,29000,0,톤28,None,https://gift.kakao.com/brand/10336
3,30,ONFJdkVFSzY,https://gift.kakao.com/product/6218926,카카오 선물하기,https://gift.kakao.com/product/6218926,{'product_name': ' N1 DE CHANEL 레드 까멜리아 립 앤 치크...,N1 DE CHANEL 레드 까멜리아 립 앤 치크 밤,63000,63000,0,샤넬,None,https://gift.kakao.com/brand/10813
4,31,kMHj6Zv4bW8,https://gift.kakao.com/product/7695898,카카오 선물하기,https://gift.kakao.com/product/7695898,"{'product_name': ' [단독/리본포장] ""푸른 텍스처+애프터배스 향"" ...","[단독/리본포장] ""푸른 텍스처+애프터배스 향"" 블루 웨이브 바디 오일 120ML",33000,33000,0,센녹,None,https://gift.kakao.com/brand/15661
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,57,YDgzXvhRkP4,https://product.29cm.co.kr/catalog/1759079?sou...,29cm,https://product.29cm.co.kr/catalog/1759079,"{'product_name': '오션 스트라이프 파자마 [여성] 3C', 'pric...",오션 스트라이프 파자마 [여성] 3C,78000,54600,30,슬리피캣,"GOOD SLEEP, GOOD DAY.",https://shop.29cm.co.kr/brand/9809
80,58,YDgzXvhRkP4,https://product.29cm.co.kr/catalog/1759102?sou...,29cm,https://product.29cm.co.kr/catalog/1759102,"{'product_name': '오션 스트라이프 파자마 [남성] 3C', 'pric...",오션 스트라이프 파자마 [남성] 3C,78000,54600,30,슬리피캣,"GOOD SLEEP, GOOD DAY.",https://shop.29cm.co.kr/brand/9809
81,60,YDgzXvhRkP4,https://product.29cm.co.kr/catalog/1803839?sou...,29cm,https://product.29cm.co.kr/catalog/1803839,"{'product_name': 'TWIN MICRO SILVER', 'price':...",TWIN MICRO SILVER,129000,110295,15,아호르,ahore는 A Harmony Of Rhythm & Essential의 약자로 흥미...,https://shop.29cm.co.kr/brand/16355
82,65,cMcpE-3re8c,https://product.29cm.co.kr/catalog/874500,29cm,https://product.29cm.co.kr/catalog/874500,"{'product_name': 'AT아이웨어머그오렌지10x11.5', 'price'...",AT아이웨어머그오렌지10x11.5,77000,69300,10,셀레티,이탈리아만의 혁신적인 아이디어로 예술과 일상이 어우러진 라이프스타일 브랜드,https://shop.29cm.co.kr/brand/11621


In [333]:
# 쇼핑몰 내 상품 id
product_id_inshop = []
for url in totProductInfo['shortURL']:
    pid = url.split('/')[-1]
    product_id_inshop.append(pid)

totProductInfo['product_id_inshop'] = product_id_inshop
totProductInfo.columns = ['product_idx', 'video_id', 'product_url', 'shoppingmall', 'short_url',
       'product_fullinfo', 'product_name', 'original_price', 'current_price', 'discount_rate', 
       'brand_name', 'brand_description', 'brand_url_inshop',
       'product_id_inshop']
totProductInfo = totProductInfo[['product_idx', 'video_id', 'product_id_inshop', 'product_url', 'shoppingmall', 'short_url',
                                 'product_name', 'original_price', 'current_price', 'discount_rate',
                                 'brand_name', 'brand_description', 'brand_url_inshop', 'product_fullinfo']]
totProductInfo

,product_idx,video_id,product_id_inshop,product_url,shoppingmall,short_url,product_name,original_price,current_price,discount_rate,brand_name,brand_description,brand_url_inshop,product_fullinfo
0,24,fflimV2pIL8,6252042,https://gift.kakao.com/product/6252042,카카오 선물하기,https://gift.kakao.com/product/6252042,[리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 한 번에' 휩드 비건 팩클렌저 1...,22700,22700,0,휩드,None,https://gift.kakao.com/brand/14844,{'product_name': ' [리본쇼핑백&헤어스크런치 증정] '팩과 클렌징을 ...
1,28,ONFJdkVFSzY,6784177,https://gift.kakao.com/product/6784177,카카오 선물하기,https://gift.kakao.com/product/6784177,월레스와 그로밋 일회용 필름 카메라 - 화려한 외출,27900,27900,0,콜리,None,https://gift.kakao.com/brand/14472,{'product_name': ' 월레스와 그로밋 일회용 필름 카메라 - 화려한 외...
2,29,ONFJdkVFSzY,5627132,https://gift.kakao.com/product/5627132,카카오 선물하기,https://gift.kakao.com/product/5627132,[단독+포장] 바디오일 & 샌달우드 괄사&에센셜오일 세트 / 3종 택1,29000,29000,0,톤28,None,https://gift.kakao.com/brand/10336,{'product_name': ' [단독+포장] 바디오일 & 샌달우드 괄사&에센셜오...
3,30,ONFJdkVFSzY,6218926,https://gift.kakao.com/product/6218926,카카오 선물하기,https://gift.kakao.com/product/6218926,N1 DE CHANEL 레드 까멜리아 립 앤 치크 밤,63000,63000,0,샤넬,None,https://gift.kakao.com/brand/10813,{'product_name': ' N1 DE CHANEL 레드 까멜리아 립 앤 치크...
4,31,kMHj6Zv4bW8,7695898,https://gift.kakao.com/product/7695898,카카오 선물하기,https://gift.kakao.com/product/7695898,"[단독/리본포장] ""푸른 텍스처+애프터배스 향"" 블루 웨이브 바디 오일 120ML",33000,33000,0,센녹,None,https://gift.kakao.com/brand/15661,"{'product_name': ' [단독/리본포장] ""푸른 텍스처+애프터배스 향"" ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,57,YDgzXvhRkP4,1759079,https://product.29cm.co.kr/catalog/1759079?sou...,29cm,https://product.29cm.co.kr/catalog/1759079,오션 스트라이프 파자마 [여성] 3C,78000,54600,30,슬리피캣,"GOOD SLEEP, GOOD DAY.",https://shop.29cm.co.kr/brand/9809,"{'product_name': '오션 스트라이프 파자마 [여성] 3C', 'pric..."
80,58,YDgzXvhRkP4,1759102,https://product.29cm.co.kr/catalog/1759102?sou...,29cm,https://product.29cm.co.kr/catalog/1759102,오션 스트라이프 파자마 [남성] 3C,78000,54600,30,슬리피캣,"GOOD SLEEP, GOOD DAY.",https://shop.29cm.co.kr/brand/9809,"{'product_name': '오션 스트라이프 파자마 [남성] 3C', 'pric..."
81,60,YDgzXvhRkP4,1803839,https://product.29cm.co.kr/catalog/1803839?sou...,29cm,https://product.29cm.co.kr/catalog/1803839,TWIN MICRO SILVER,129000,110295,15,아호르,ahore는 A Harmony Of Rhythm & Essential의 약자로 흥미...,https://shop.29cm.co.kr/brand/16355,"{'product_name': 'TWIN MICRO SILVER', 'price':..."
82,65,cMcpE-3re8c,874500,https://product.29cm.co.kr/catalog/874500,29cm,https://product.29cm.co.kr/catalog/874500,AT아이웨어머그오렌지10x11.5,77000,69300,10,셀레티,이탈리아만의 혁신적인 아이디어로 예술과 일상이 어우러진 라이프스타일 브랜드,https://shop.29cm.co.kr/brand/11621,"{'product_name': 'AT아이웨어머그오렌지10x11.5', 'price'..."


In [335]:
totProductInfo.to_csv('./productinfo/product(1).csv')